### Import all necessary libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from calendar import monthrange
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import yfinance as yf
%matplotlib inline
import warnings
warnings.simplefilter("ignore")

### Get stock data from Yahoo Finance
I have chosen my favourite stock here Bajaj Finance. It is a high beta stocks and thus provides opportunites for short term trading.
Each entry represents 1 day staring from 1 Jan 2011

In [5]:
df = pd.read_csv('/Users/ankit/Downloads/fo_mktlots.csv')
top_stocks=[k.strip()+'.NS' for k in df['SYMBOL    '][3:]]
all_ticker_df = pd.DataFrame()
for ticker in top_stocks:
    tickerData = yf.Ticker(ticker)
    tickerDf = tickerData.history(period='1d', start='2020-12-1')
    tickerDf['Ticker']=ticker
    all_ticker_df=pd.concat([all_ticker_df,tickerDf])

In [8]:
df = all_ticker_df

In [9]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker
Date,,,,,,,,
2020-12-01,2384.00,2508.85,2349.05,2463.00,2001392,0.0,0,APOLLOHOSP.NS
2020-12-02,2474.70,2485.05,2375.45,2411.55,1611325,0.0,0,APOLLOHOSP.NS
2020-12-03,2501.00,2501.05,2371.90,2389.90,1745381,0.0,0,APOLLOHOSP.NS
2020-12-04,2400.00,2445.00,2400.00,2434.65,1219317,0.0,0,APOLLOHOSP.NS
2020-12-07,2434.65,2468.55,2418.05,2432.70,778998,0.0,0,APOLLOHOSP.NS
...,...,...,...,...,...,...,...,...
2021-01-19,328.60,341.00,327.60,339.10,2541204,0.0,0,TORNTPOWER.NS
2021-01-20,341.80,343.70,331.55,333.55,2093618,0.0,0,TORNTPOWER.NS
2021-01-21,335.60,338.35,326.10,327.00,1803090,0.0,0,TORNTPOWER.NS


In [6]:
### Bollinger Bands
window = 21
no_of_std = 2

#Calculate rolling mean and standard deviation using number of days set above
rolling_mean = df['Close'].rolling(window).mean()
rolling_std = df['Close'].rolling(window).std()
rolling_vol = df['Volume'].rolling(window).mean()

#create two new DataFrame columns to hold values of upper and lower Bollinger bands
df['Rolling Mean'] = rolling_mean
df['Bollinger High'] = rolling_mean + (rolling_std * no_of_std)
df['Bollinger Low'] = rolling_mean - (rolling_std * no_of_std)
df['Rolling Vol'] = rolling_vol

KeyError: 'Close'

In [ ]:
#### RSI
window = 14
no_of_std = 2
df['Prev_Close'] = df['Close'].shift(1)
df['GL'] = (df['Close']-df['Prev_Close'])/df['Prev_Close']
df['rsi']=None
for row in range(df.shape[0]):
    tmp = df['GL'].iloc[row-14:row]
    avg_gain = (tmp[tmp>0].sum())/14
    avg_loss = abs(tmp[tmp<0].sum())/14
    df['rsi'].iloc[row] = 100-100/(1+avg_gain/avg_loss)
#     avg_gain = avg_gain*13 + max(df['GL'].iloc[row], 0)
#     avg_loss = (avg_loss)*13 - min(df['GL'].iloc[row], 0)
#     rs = avg_gain/avg_loss
#     df['rsi'].iloc[row] = 100-100/(1+rs)

In [ ]:
#### macd
df['macd'] = df['Close'].ewm(span=12, adjust=False).mean()-df['Close'].ewm(span=26, adjust=False).mean()
df['signal'] = df['macd'].ewm(span=9, adjust=False).mean()
df['macd_hist'] = df['macd']-df['signal']

NameError: name 'bollinger_bands' is not defined

### Bollinger Bands
Bollinger Bands are a technical analysis tool developed by John Bollinger for generating oversold or overbought signals.
There are three lines that compose Bollinger Bands: A simple moving average (middle band) and an upper and lower band.
The upper and lower bands are typically 2 standard deviations +/- from a 20-day simple moving average, but can be modified.

### Relative Strength Index (RSI)
The relative strength index (RSI) is a momentum indicator used in technical analysis that measures the magnitude of recent price changes to evaluate overbought or oversold conditions in the price of a stock or other asset. The RSI is displayed as an oscillator (a line graph that moves between two extremes) and can have a reading from 0 to 100. The indicator was originally developed by J. Welles Wilder Jr. and introduced in his seminal 1978 book, "New Concepts in Technical Trading Systems."

Traditional interpretation and usage of the RSI are that values of 70 or above indicate that a security is becoming overbought or overvalued and may be primed for a trend reversal or corrective pullback in price. An RSI reading of 30 or below indicates an oversold or undervalued condition.

### MACD
Moving Average Convergence Divergence (MACD) is a trend-following momentum indicator that shows the relationship between two moving averages of a security’s price. The MACD is calculated by subtracting the 26-period Exponential Moving Average (EMA) from the 12-period EMA.

The result of that calculation is the MACD line. A nine-day EMA of the MACD called the "signal line," is then plotted on top of the MACD line, which can function as a trigger for buy and sell signals. Traders may buy the security when the MACD crosses above its signal line and sell—or short—the security when the MACD crosses below the signal line. Moving Average Convergence Divergence (MACD) indicators can be interpreted in several ways, but the more common methods are crossovers, divergences, and rapid rises/falls.

In [ ]:
#### return to risk ratio - 1, 2, and 3 month
# df['rallyGain']=0
# for row in range(df.shape[0]-1):
#     gain=0
#     if df['GL'].iloc[row+1] > 0:
#         for r in range(row+1,df.shape[0]):
#             if df['GL'].iloc[r] < 0:
#                 break
#             gain+=df['GL'].iloc[r]
#     df['rallyGain'].iloc[row] = round(gain,2)
### Lowest price in next n days - 20, 40, 60
for n in [1,2,3]:
    df['ret2risk'+str(n)] = None
    for row in range(df.shape[0]):
        tmp = df['Close'].iloc[row+1:row+20*n+1]
        if len(tmp) > 20*n-1:
            low = min(tmp)
            high = max(tmp)
            if low < df['Close'][row]:
                ret2risk = (high - df['Close'][row])/(df['Close'][row] - low)
            else:
                ret2risk = (high - df['Close'][row] + low)/(low - df['Close'][row])
            df['ret2risk'+str(n)].iloc[row] = int(ret2risk > 1)

In [ ]:
import datetime
import calendar
def prev_weekday(d, weekday):
    days_ahead = d.weekday() - weekday
    if days_ahead <= 0:
        days_ahead += 7
    return d - datetime.timedelta(days_ahead)
def days2expiry(dt):
    yr = dt.year
    mth = dt.month
    d = datetime.date(yr,mth,monthrange(yr, mth)[1])
    expiry = prev_weekday(d, 3)
    return (expiry-dt.date()).days

In [ ]:
#### Feature Engineering
df['BreakHigh'] = 0
df['BreakClose'] = 0
df['BreakHigh'][df['High']>df['Bollinger High']] = 1
df['BreakClose'][df['Close']>df['Bollinger High']] = 1

df['BreakLow'] = 0
df['BreakBear'] = 0
df['BreakLow'][df['Low']<df['Bollinger Low']] = 1
df['BreakBear'][df['Close']<df['Bollinger Low']] = 1

df['overbought']=0
df['overbought'][df['rsi']>70]=1
df['oversold']=0
df['oversold'][df['rsi']<30]=1

df['MACDmoreSignal'] = 0
df['MACDmoreSignal'][df['macd']>df['signal']] = 1

df['OperatorEntry']=0
df['OperatorEntry'][df['Volume']>df['Rolling Vol']]=1

## HmO, HmC, HmL, OmL, CmL
df['HmO'] = (df['High'] - df['Open'])/df['Open']
df['HmC'] = (df['High'] - df['Close'])/df['Open']
df['HmL'] = (df['High'] - df['Low'])/df['Open']
df['OmL'] = (df['Open'] - df['Low'])/df['Open']
df['CmL'] = (df['Close'] - df['Low'])/df['Open']


### SMAs 20, 50, 100, 200
df['sma_20'] = df['Close'].rolling(20).mean()
df['sma_50'] = df['Close'].rolling(50).mean()
df['sma_100'] = df['Close'].rolling(100).mean()
df['sma_200'] = df['Close'].rolling(200).mean()
df['20g50'] = df['sma_20'] > df['sma_50']
df['20g100'] = df['sma_20'] > df['sma_100']
df['20g200'] = df['sma_20'] > df['sma_200']
df['Cg20'] = df['Close'] > df['sma_20']
df['Cg50'] = df['Close'] > df['sma_50']
df['Cg100'] = df['Close'] > df['sma_100']
df['Cg200'] = df['Close'] > df['sma_200']

df['20g50'] = df['20g50'].apply(lambda x:int(x))
df['20g100'] = df['20g100'].apply(lambda x:int(x))
df['20g200'] = df['20g200'].apply(lambda x:int(x))
df['Cg20'] = df['Cg20'].apply(lambda x:int(x))
df['Cg50'] = df['Cg50'].apply(lambda x:int(x))
df['Cg100'] = df['Cg100'].apply(lambda x:int(x))
df['Cg200'] = df['Cg200'].apply(lambda x:int(x))

### DOW, DFME, DOM
df['date'] = df.index
df['dow'] = df.index.dayofweek
df['DFME'] = df['date'].apply(lambda x:days2expiry(x))
df['DOM'] = df['date'].apply(lambda x:x.day)

In [ ]:
df.columns

In [ ]:
df=df[['macd_hist', 'ret2risk1',
       'ret2risk2', 'ret2risk3', 'BreakHigh', 'BreakClose', 'BreakLow',
       'BreakBear', 'overbought', 'oversold', 'MACDmoreSignal',
       'OperatorEntry', 'HmO', 'HmC', 'HmL', 'OmL', 'CmL', '20g50', '20g100', '20g200', 'Cg20', 'Cg50',
       'Cg100', 'Cg200', 'dow', 'DFME', 'DOM']]

In [ ]:
df=df.dropna()

In [ ]:
col = [k for k in df.columns if k not in ['ret2risk1', 'ret2risk2', 'ret2risk3']]

In [ ]:
targets = ['ret2risk1', 'ret2risk2', 'ret2risk3']
models = [LogisticRegression(random_state=0), RandomForestClassifier(random_state=1)]
for target in targets:
    X = df[col].iloc[:1400]
    y = df[target].iloc[:1400].astype('int')
    print("Target: {}".format(target))
    for model in models:
        clf = model.fit(X, y)
        y_pred = clf.predict(df[col].iloc[1400:])
        tn, fp, fn, tp = confusion_matrix(df['ret2risk2'].iloc[1400:].astype('int'), y_pred).ravel()
        p, r = tp/(tp+fp), tp/(tp+fn)
        print("Model: {}".format(model))
        print("Test Set Results:")
        print ("Precision: {}, Recall :{}, F1 :{}".format(p, r, 2*p*r/(p+r)))
        y_pred = clf.predict(df[col].iloc[:1400])
        tn, fp, fn, tp = confusion_matrix(df['ret2risk2'].iloc[:1400].astype('int'), y_pred).ravel()
        p, r = tp/(tp+fp), tp/(tp+fn)
        print("Train Set Results:")
        print ("Precision: {}, Recall :{}, F1 :{}".format(p, r, 2*p*r/(p+r)))
        print('\n')

In [ ]:
df.columns